# Experiment Tracking
- In this lesson, we'll learn how to keep track of every experiment you run using [MLFlow](https://mlflow.org/)
- MLFlow has other components that help create production ML code. I highly encourage you to spend sometime going through the [MLFlow docs](https://mlflow.org/docs/latest/quickstart.html)

In [18]:
# Ensure the MLFlow library is installed
#!pip install -U mlflow

# Initialize MLFlow
1. Open a new terminal window
1. Activate the `dsi-sg` mamba/conda environment
1. Navigate to the `12.01-mlops/solution-code` lesson folder
1. Now run the below code from this terminal window. Ensure you're inside the `solution-code` folder!: `mlflow ui --backend-store-uri sqlite:///mlflow.db`
1. The above code starts a small webserver running the MLFlow application on your computer. It uses SQLite to store the experiment information in a database
1. You can open the mlflow UI by opening this link in your browser: http://localhost:5000/

<img src="../images/01_01_MLFlow_UI.png" width="1000">

In [19]:
#pip install -U mlflow

In [20]:
#!pip3 install mlflow

In [21]:
#conda install -c conda-forge mlflow

In [6]:
# Connect this Jupyter notebook to the running MLFlow server 
import mlflow

In [7]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [8]:
# Set the name of the experiment we're running in this notebook
# MLFlow will connect to an existing experiment or create a new one if the experiment is not already present
mlflow.set_experiment("grad-school-admission")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='grad-school-admission', tags={}>

In [9]:
# Start automatically logging experiment details to MLFlow
mlflow.autolog()

<img src="../images/01_02_MLFlow_new_experiment.png" width="1000">

# Normal ML code

In [10]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import get_scorer

2022/09/15 19:42:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [11]:
# Read in the data
admissions = pd.read_csv('../data/grad_admissions.csv')

# Drop NA
admissions.dropna(inplace=True)

print(admissions.shape)
admissions.head()

(3970, 3)


,admit,gre,gpa
0,0,380.0,2.915018
1,1,660.0,4.044540
2,1,800.0,4.950714
3,1,640.0,3.921994
4,0,520.0,2.069878


In [12]:
# Split features & response, define training & testing sets
X = admissions.drop(columns=['admit']) # feature matrix
y = admissions['admit'] # response vector

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 50)

In [13]:
# Fit a Logistic Regression Model on raw data
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

2022/09/15 19:42:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'fb7492639e83444cbadea8e631000f2c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/09/15 19:42:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Cathy\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


LogisticRegression()

<img src="../images/01_03_MLFlow_run.png" width="1000">

In [14]:
# Scale and fit Logistic Regression Model in a Pipeline
sc = StandardScaler()
logreg = LogisticRegression()

pipe_model = Pipeline(
    [
        ("scaling", sc),
        ("lr", logreg)
    ]
)

pipe_model.fit(X_train, y_train)

2022/09/15 19:42:22 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f6a3ef28210a4181aad049e68afffb35', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Pipeline(steps=[('scaling', StandardScaler()), ('lr', LogisticRegression())])

<img src="../images/01_04_MLFlow_sort.png" width="1000">

## Slighlty updated ML code
- MLFlow autologging only logs the training metrics by default

<img src="../images/01_05_MLFlow_train_metrics.png" width="400">

- To log test metrics as well, we need to just calculate any test metric we want inside a `with mlflow.start_run():` block as below:

    ```
    # Previously
    pipe_model.fit(X_train, y_train)
    
    # New
    with mlflow.start_run():
        pipe_model.fit(X_train, y_train)
        
        # Calculate any test metric we want to log here
        
    ```

In [15]:
# Scale and fit Logistic Regression Model and log test metrics as well
sc = StandardScaler()
logreg = LogisticRegression()

pipe_model = Pipeline(
    [
        ("scaling", sc),
        ("lr", logreg)
    ]
)

with mlflow.start_run():
    pipe_model.fit(X_train, y_train)
    
    # These metrics get logged automatically by just calculating them
    for metric in ['accuracy', 'precision', 'recall', 'f1']:
        print(f'Test {metric}: {get_scorer(metric)(pipe_model, X_test, y_test)}')
    
    # We can also log metrics manually to the same run
    mlflow.log_metrics({'roc_auc_score_X_test': get_scorer('roc_auc')(pipe_model, X_test, y_test)})

Test accuracy: 0.904330312185297
Test precision: 0.8368055555555556
Test recall: 0.8339100346020761
Test f1: 0.8353552859618718


<img src="../images/01_06_MLFlow_all_metrics.png" width="400">

## Hyperparameter tuning is exactly the same!

In [16]:
# Logging runs of hyperparameter tuning.
sc = StandardScaler()
logreg = LogisticRegression()

pipe_model = Pipeline(
    [
        ("scaling", sc),
        ("lr", logreg)
    ]
)

params = {
    'lr__solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'lr__penalty': ['l2'],
    'lr__C': [100, 10, 1.0, 0.1, 0.01]
}

gridsearch_model = GridSearchCV(pipe_model, 
                                params, 
                                cv=5,
                                verbose=1, 
                                n_jobs=-1)

with mlflow.start_run():
    gridsearch_model.fit(X_train, y_train)
    
    for metric in ['accuracy', 'precision', 'recall', 'f1']:
        print(f'Test {metric}: {get_scorer(metric)(gridsearch_model, X_test, y_test)}')
        
    mlflow.log_metrics({'roc_auc_score_X_test': get_scorer('roc_auc')(gridsearch_model, X_test, y_test)})

Fitting 5 folds for each of 15 candidates, totalling 75 fits


2022/09/15 19:42:48 INFO mlflow.sklearn.utils: Logging the 5 best runs, 10 runs will be omitted.


Test accuracy: 0.9063444108761329
Test precision: 0.8426573426573427
Test recall: 0.8339100346020761
Test f1: 0.8382608695652175


In [17]:
gridsearch_model.best_params_

{'lr__C': 0.1, 'lr__penalty': 'l2', 'lr__solver': 'newton-cg'}

### Hyperparameter runs are grouped together for convenience
<img src="../images/01_07_MLFlow_hyperparam.png" width="1000">

### The parameters and the metrics of the best model are all logged
<img src="../images/01_08_MLFlow_hyperparam_best_params.png" width="600">

### The best model is also saved to a file so we can deploy it easily
<img src="../images/01_09_MLFlow_hyperparam_best_model.png" width="1000">

# Download model for deployment
- After we've run all our experiments, we can choose a model to prepare for deployment
- We can use the path of the run to download the required artifacts such as saved model, requirements.txt, etc
- This is all the info we need to deploy this model!

<img src="../images/01_10_MLFlow_trained_model.png" width="1000">

In [18]:
from mlflow.artifacts import download_artifacts


In [20]:

# Download the desired model from MLFlow to local directory
# Get the URL by following instructions in above image
full_path = './mlruns/1/dd88565752c9428dbaa71955e4f34e99/artifacts/best_estimator'
download_artifacts(full_path, dst_path='.')

'C:\\Users\\Documents\\GA\\my_materials\\12.01-mlops\\solution-code\\best_estimator'

In [21]:
# Load the downloaded model
import mlflow.pyfunc

model = mlflow.pyfunc.load_model(model_uri="./best_estimator")

In [22]:
# Run predictions
y_pred = model.predict(X_test)

# Print first five predictions
y_pred[:5]

array([0, 0, 0, 1, 0], dtype=int64)

# Cleanup

## To stop and restart MLFlow server
- To stop the MLFlow server, press `Ctrl + C` in the terminal window that's running the MLFlow server.
- You can restart the MLFlow server ***IN THE SAME FOLDER*** by running `mlflow ui --backend-store-uri sqlite:///mlflow.db` again and all your past experiments will still show up!

## To totally remove all MLFlow files to start from scratch
1. ***WARNING!*** Doing this will wipe out all your MLFlow experiment runs and saved models and there's no way to recover your data!
1. Stop MLFlow server by pressing `Ctrl + C` in the terminal window that's running the MLFlow server
1. Delete the `mlruns` directory
1. Delete the `mlflow.db` file

# Conclusion
In this lesson, we
1. Installed and started up an MLFlow server on our local computer
1. Used MLFlow autologging to log experiment runs to the MLFlow server
1. Saw how we can log additional metrics such as test metrics and hyperparameter searches
1. Downloaded the final version of the model that we want to deploy